In [1]:
!pip install -U "tensorflow-text==2.8.*"
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets,layers,models
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding,Bidirectional
import tensorflow_hub as hub
import tensorflow_text as text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 7.6 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.9/497.9 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 30.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.4 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 32.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorboard
 

2022-12-26 13:10:09.560048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-26 13:10:09.560098: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
"""# download glove and unzip it in Notebook.
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip.1"""

'# download glove and unzip it in Notebook.\n!wget http://nlp.stanford.edu/data/glove.6B.zip\n!unzip glove*.zip.1'

In [3]:
def embedding_for_vocab(filepath, word_index,embedding_dim):
    vocab_size = len(word_index) + 1
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,embedding_dim))
    with open(filepath,encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix_vocab

In [4]:
def plot_history(history):
    plt.plot(history.history['precision'])
    plt.plot(history.history['val_precision'])
    plt.title('model precision')
    plt.ylabel('Precision')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def plot_train(train_images,class_names,train_labels):
    plt.figure(figsize=(20,10))
    for i in range(5):
        plt.subplot(5,2,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i],cmap=plt.cm.binary)
        # The CIFAR labels happen to be arrays,# which is why you need the extra index
        plt.xlabel(class_names[train_labels[i]])
        plt.show()

In [5]:
datas = pd.read_csv("/kaggle/input/goemotions/GoEmotions.csv")
data = datas.drop(['id','author','subreddit','link_id','parent_id','rater_id','created_utc'],axis=1)
data = data[data['example_very_unclear']==False]
data = data.drop_duplicates('text')
data = data[data["neutral"]==0].drop(['neutral','example_very_unclear'],axis=1)
data.describe(include="all")

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
count,41927,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,...,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000,41927.000000
unique,41927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,That game hurt.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.116250,0.064016,0.053474,0.084289,0.116822,0.038424,0.048251,0.066974,0.024185,...,0.052591,0.056527,0.011663,0.058387,0.008443,0.056885,0.008873,0.017030,0.044148,0.036444
std,NaN,0.320528,0.244784,0.224979,0.277825,0.321212,0.192220,0.214298,0.249979,0.153625,...,0.223219,0.230939,0.107366,0.234477,0.091499,0.231625,0.093777,0.129383,0.205427,0.187395
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
data.head()

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Man I love reddit.,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,Right? Considering it’s such an important docu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"He isn't as big, but he's still quite popular....",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,That's crazy; I went to a super [RELIGION] hig...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#data[(data['disapproval']==data['joy']) & (data['joy']==1)] #some texts have multiple emotions

In [8]:
"""data = data[np.sum(datadrop('text',axis=1),axis=1)<=1]
data.describe()"""

"data = data[np.sum(datadrop('text',axis=1),axis=1)<=1]\ndata.describe()"

In [9]:
#tokenize
tokenizer = Tokenizer(
    num_words=None,
    lower = False,
    split=' '
)
abst = data['text'].values
tokenizer.fit_on_texts(abst)
sequences = tokenizer.texts_to_sequences(abst)

#pad sequences so they all have same lengths
sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences,
    maxlen=max([len(i) for i in sequences]),padding='post')
sequences[0]

array([ 83, 140, 537,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [10]:
text, labels = data['text'], data.drop('text',axis=1).values
features = np.array([item for item in [seq for seq in sequences]])
features[0]

array([ 83, 140, 537,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [11]:
X_train, X_val, Y_train, Y_val = train_test_split(features, labels, test_size = 0.2, random_state=23)
X_train.shape

(33541, 34)

In [12]:
Y_train[0]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [13]:
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
#build model
def create_model(training_length,num_words,embedding_matrix=[]):
   
    # Embedding layer
    if len(embedding_matrix)>0:
        model = Sequential()
        model.add(
        Embedding(input_dim=num_words,
                  input_length = training_length,
                  output_dim=300,
                  weights=[embedding_matrix],
                  trainable=False,
                  mask_zero=False))
            # Recurrent layer
        model.add(Dropout(0.2))
        model.add(Bidirectional(CuDNNLSTM(128, return_sequences=False)))
        # Fully connected layer
        model.add(Dense(64, activation='tanh'))
        model.add(Dropout(0.3))

        # Output layer
        model.add(Dense(27, activation='sigmoid'))
        # Masking layer for pre-trained embeddings
        #model.add(Masking(mask_value=0.0))
    else:
        bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
        bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1")

        text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='inputs')
        preprocessed_text = bert_preprocess(text_input)
        outputs = bert_encoder(preprocessed_text)
        l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
        l = tf.keras.layers.Dense(27, activation='sigmoid', name="output")(l)
        model = tf.keras.Model(inputs=[text_input], outputs = [l])


    return model

In [14]:
"""embedding_matrix_vocab = embedding_for_vocab('./glove.6B.300d.txt', tokenizer.word_index,300)"""

"embedding_matrix_vocab = embedding_for_vocab('./glove.6B.300d.txt', tokenizer.word_index,300)"

In [15]:
vocab_size = len(tokenizer.word_index)+1

In [16]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_precision', mode='max', verbose=1, patience=10)

In [17]:
x_train, x_val, y_train, y_val = train_test_split(data['text'],data.drop(['text'],axis=1),  test_size = 0.2, random_state=23)

In [18]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3442982412109294890
xla_global_id: -1
]


2022-12-26 13:10:20.890655: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-26 13:10:20.892490: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:
2022-12-26 13:10:20.892522: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-26 13:10:20.892559: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cef69629effa): /proc/driver/n

In [19]:
strategy = tf.distribute.MirroredStrategy(['gpu:0','gpu:1'])
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.with_options(options).batch(1000).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.with_options(options).batch(1000)

Number of devices: 2


In [20]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 27), dtype=tf.int64, name=None))>

In [ ]:
# Open a strategy scope.
with strategy.scope():
  # Everything that creates variables should be under the strategy scope.
  # In general this is only model construction & `compile()`.
    model = create_model(34,vocab_size)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=tf.keras.metrics.Precision(name='precision'))
    model.summary()
    history = model.fit(train_dataset,epochs=5,
                    validation_data=val_dataset)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer_9 (KerasLayer)     {'input_word_ids':   0           ['inputs[0][0]']                 
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [ ]:
plot_history(history)

In [26]:
model = tf.saved_model.load('/kaggle/input/goemon')

OSError: SavedModel file does not exist at: /kaggle/input/goemon/{saved_model.pbtxt|saved_model.pb}

In [ ]:
pred = model.predict(X_val)

In [ ]:
pred[0]

In [ ]:
def prob(x):
    if x>0.5:
        return 1
    return 0
map(prob,pred[0])

In [141]:
pred = [list(map(prob,i)) for i in pred]

In [142]:
idx = tokenizer.index_word
phrase = ' '.join(idx[w] for w in X_val[0] if w!=0)
print(phrase,Y_val[0],pred[0])

NAME completely unpredictable Bunch of them were standing together in the middle of the ring lol [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [167]:
from sklearn.metrics import confusion_matrix
confusion_matrix([np.argmax(i) for i in Y_train],pred)

array([[2243,   60,  321,   99,   89,    8,    1,   21,    2,    2,   25,
           0,    0,   27,    2,   46,    0,   12,   21,    0,   11,    2,
          20,    0,    0,    7,   11],
       [   8, 1292,   28,    9,    2,    0,    0,    2,    0,    0,    5,
           0,    0,    3,    0,    3,    0,    3,    5,    0,    3,    1,
           1,    0,    1,    1,    0],
       [   6,    6, 1026,   12,    2,    2,    1,    2,    2,    0,    4,
           2,    0,    4,    1,   13,    1,    0,    3,    0,    0,    0,
           3,    0,    0,    2,    0],
       [   7,   11,   30, 1583,    4,    3,    0,    3,    0,    2,    2,
           1,    3,    1,    0,    1,    0,    2,    1,    0,    4,    0,
           0,    0,    0,    4,    0],
       [  30,   22,   30,    8, 2222,    6,    0,    2,    4,    6,   22,
           1,    2,    8,    1,   12,    1,   10,   11,    0,   26,    3,
          15,    2,    1,    4,    1],
       [   6,    4,    8,    2,    1,  691,    0,    0,    0,    

In [ ]:
modele.save("goemotions.h5")